# Setup Inicial

Este notebook se utiliza para cargar el dataset obtenido de Kaggle en ElasticSearch. Este paso en este caso es un poco forzado, porque se asume que en el escenario real los datos ya estarán disponibles en la base de datos ElasticSearch y se tomarán directamente de ahí.
Luego de la carga, se deben realizar algunas acciones de configuración/verificación en Kibana.

In [1]:
ELASTICSEARCH_HOST = 'elasticsearch'
ELASTICSEARCH_PORT = 9200
DATASET_ORIGINAL_FILENAME_PATH = "/data/credit_fraud/creditcard.csv"
DATASET_MODIFIED_FILENAME_PATH = "/data/credit_fraud/creditcard_es.csv"
ELASTICSEARCH_DATASET_INDEX = 'creditfraud-1'

## 1. Modificación del dataset

Se harán dos modificaciones:
1. El dataset original contiene el tiempo relativo en segundos. Se asignará un tiempo inicial arbitrario para poder hacer búsquedas por intervalo de tiempo.
2. Se agregará una columna con un identificador artificial utilizando el número de fila del CSV para luego separar el dataset en entrenamiento y validación utilizando este identificador.

In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
# Se elige una fecha arbitraria de inicio
STARTING_DATE = datetime.datetime(2020, 10, 15)

In [4]:
col_names = [ "time" ]
col_names.extend(["v{}".format(x) for x in range(1,29)])
col_names.append( "amount" )
col_names.append( "class" )
df = pd.read_csv(DATASET_ORIGINAL_FILENAME_PATH,skiprows=1, names=col_names)
df['row_id'] = np.arange(len(df))
epoch = datetime.datetime.fromtimestamp(0)
df['time'] = df['time'].apply(lambda x: (STARTING_DATE + datetime.timedelta(seconds=x)-epoch).total_seconds() )

df.to_csv(DATASET_MODIFIED_FILENAME_PATH,index=False)

In [5]:
modified_col_names = col_names.copy()
modified_col_names.append('row_id')
df2 = pd.read_csv(DATASET_MODIFIED_FILENAME_PATH,skiprows=1,index_col=0, names=modified_col_names)
df2.head(10)

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v22,v23,v24,v25,v26,v27,v28,amount,class,row_id
time,,,,,,,,,,,,,,,,,,,,,
1.602720e+09,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,0
1.602720e+09,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,1
1.602720e+09,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,2
1.602720e+09,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,3
1.602720e+09,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,4
1.602720e+09,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0,5
1.602720e+09,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,-0.099254,...,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0,6
1.602720e+09,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,...,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0,7
1.602720e+09,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,...,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0,8


In [6]:
total_rows = len(df2)
print(total_rows)

284807


## 2. Carga por bulk a ElasticSearch

Antes de cargar los datos en ElasticSearch, registrar el siguiente template en la [consola de Kibana](http://localhost:5601/app/dev_tools#/console)
Este template fuerza la intepretación del campo time como timestamp para que pueda ser indexado y aplicado a las búsquedas.

~~~json
PUT _template/creditfraud
{
  "index_patterns": ["creditfraud-*"],
  "mappings": {
    "_source": {
        "enabled": true
    },
    "properties": {
        "time": {
            "type": "date",
            "format": "epoch_second"
         }
    }
  }
}
~~~

In [7]:
input("Cargar Index Template antes de continuar")

Cargar Index Template antes de continuar


''

El siguiente paso carga los datos en ElasticSearch utilizando la API de bulk (tarda unos minutos).

In [8]:
from elasticsearch import helpers, Elasticsearch
import csv

es = Elasticsearch(f"http://{ELASTICSEARCH_HOST}:{ELASTICSEARCH_PORT}/")
es.indices.delete(index=ELASTICSEARCH_DATASET_INDEX, ignore=[400, 404])
with open(DATASET_MODIFIED_FILENAME_PATH) as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index=ELASTICSEARCH_DATASET_INDEX)

## 3. Verificación / visualización en Kibana

1. Conectarse a [Kibana](http://localhost:5601)
2. Cambiar al panel para agregar índices: [link](http://localhost:5601/app/management/kibana/indexPatterns)
3. Crear un índice con la expresión "creditfraud-*" y usar el campo "time" para indexado temporal.

In [9]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

s = Search(using=es, index=ELASTICSEARCH_DATASET_INDEX)

df_from_es = pd.DataFrame([hit.to_dict() for hit in s.scan()]).astype(float)
assert len(df_from_es) == total_rows
df_from_es.head()

,time,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v22,v23,v24,v25,v26,v27,v28,amount,class,row_id
0,1.602720e+09,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0,0.0
1,1.602720e+09,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0,1.0
2,1.602720e+09,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0,2.0
3,1.602720e+09,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0,3.0
4,1.602720e+09,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0,4.0


In [10]:
df_from_es = df_from_es.sort_values(by=['time'])

In [11]:
print("Timestamp del primer registro",datetime.datetime.fromtimestamp(df_from_es['time'].iloc[0]))
print("Timestamp del último registro",datetime.datetime.fromtimestamp(df_from_es['time'].iloc[-1]))

Timestamp del primer registro 2020-10-15 00:00:00
Timestamp del último registro 2020-10-16 23:59:52
